In [1]:
import json
import string
import re
import nltk
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_set = open("C:/Users/user/Desktop/ML_Challenge/train-1.json")
test_set = open("C:/Users/user/Desktop/ML_Challenge/test.json")
train = json.load(train_set)
test = json.load(test_set)

In [3]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

In [4]:
df_train.head()

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"[A. Akbik, Laura Chiticariu, Marina Danilevsky...",ACL,2015.0,39,[Semantic role labeling],True,[Computer Science],60
1,10.18653/v1/2020.eval4nlp-1.12,One of these words is not like the other: a re...,Word embeddings are an active topic in the NLP...,"[Jesper Brink Andersen, Mikkel Bak Bertelsen, ...",EVAL4NLP,2020.0,44,[],True,[Computer Science],1
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"[Kyle Richardson, Sina Zarrieß, Jonas Kuhn]",INLG,2017.0,30,"[Natural language generation, Library (computi...",True,[Computer Science],5
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"[Rik van Noord, Johan Bos]",SemEval@ACL,2017.0,11,"[Parsing, Convolutional neural network, Text-b...",True,[Computer Science],5
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"[Elisabeth Lien, Milen Kouylekov]",IWPT,2015.0,26,"[Textual entailment, Parsing, SemEval, Semanti...",True,[Computer Science],10


In [5]:
df_test.head()

,is_open_access,doi,title,fields_of_study,abstract,year,venue,references,authors,topics
0,False,10.18653/v1/2021.findings-acl.255,On Sparsifying Encoder Outputs in Sequence-to-...,[Computer Science],Sequence-to-sequence models usually transfer a...,2021,FINDINGS,47,"[Biao Zhang, Ivan Titov, Rico Sennrich]","[Encoder, Transformer, Automatic summarization..."
1,True,10.18653/v1/2020.acl-main.200,To Pretrain or Not to Pretrain: Examining the ...,"[Computer Science, Mathematics]",Pretraining NLP models with variants of Masked...,2020,ACL,18,"[Sinong Wang, Madian Khabsa, Hao Ma]","[Supervised learning, Language model, Document..."
2,True,10.18653/v1/W18-0211,Utilization of Nganasan digital resources: a s...,[Computer Science],"According to the wide-spread belief, although ...",2018,,8,[L. Fejes],"[Rounding, Lexicon, Tracer, Body of uterus, Cl..."
3,True,10.18653/v1/N19-1413,A large-scale study of the effects of word fre...,[Computer Science],A number of psycholinguistic studies have fact...,2019,NAACL,52,[Cory Shain],"[Word lists by frequency, Text corpus, Sentenc..."
4,False,10.18653/v1/2021.semeval-1.168,Amherst685 at SemEval-2021 Task 7: Joint Model...,[Computer Science],This paper describes our submission to theSemE...,2021,SEMEVAL,17,"[Brian Zylich, Akshay Gugnani, Gabriel Brookma...",[]


In [6]:
df_test = df_test.reindex(["doi", "title", "abstract", "authors", "venue", "year", "references",
                           "topics", "is_open_access", "fields_of_study"], axis=1)

In [7]:
df_test.head()

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study
0,10.18653/v1/2021.findings-acl.255,On Sparsifying Encoder Outputs in Sequence-to-...,Sequence-to-sequence models usually transfer a...,"[Biao Zhang, Ivan Titov, Rico Sennrich]",FINDINGS,2021,47,"[Encoder, Transformer, Automatic summarization...",False,[Computer Science]
1,10.18653/v1/2020.acl-main.200,To Pretrain or Not to Pretrain: Examining the ...,Pretraining NLP models with variants of Masked...,"[Sinong Wang, Madian Khabsa, Hao Ma]",ACL,2020,18,"[Supervised learning, Language model, Document...",True,"[Computer Science, Mathematics]"
2,10.18653/v1/W18-0211,Utilization of Nganasan digital resources: a s...,"According to the wide-spread belief, although ...",[L. Fejes],,2018,8,"[Rounding, Lexicon, Tracer, Body of uterus, Cl...",True,[Computer Science]
3,10.18653/v1/N19-1413,A large-scale study of the effects of word fre...,A number of psycholinguistic studies have fact...,[Cory Shain],NAACL,2019,52,"[Word lists by frequency, Text corpus, Sentenc...",True,[Computer Science]
4,10.18653/v1/2021.semeval-1.168,Amherst685 at SemEval-2021 Task 7: Joint Model...,This paper describes our submission to theSemE...,"[Brian Zylich, Akshay Gugnani, Gabriel Brookma...",SEMEVAL,2021,17,[],False,[Computer Science]


In [8]:
df_train.shape

(9658, 11)

In [9]:
df_test.shape

(1000, 10)

In [10]:
df_train.isnull().sum()

doi                  0
title                0
abstract           159
authors              0
venue                0
year                 3
references           0
topics               0
is_open_access       0
fields_of_study    136
citations            0
dtype: int64

In [11]:
df_test.isnull().sum()

doi                 0
title               0
abstract           19
authors             0
venue               0
year                0
references          0
topics              0
is_open_access      0
fields_of_study    13
dtype: int64

In [12]:
#drop rows with missing values
df_train = df_train.dropna()
df_test = df_test.dropna()

In [13]:
df_train["authors"] = [",".join(i) for i in df_train["authors"]]
df_train["topics"] = [",".join(i) for i in df_train["topics"]]
df_train["fields_of_study"] = [",".join(i) for i in df_train["fields_of_study"]]

In [14]:
df_train = df_train.replace(r'^\s*$', np.nan, regex=True)

In [15]:
df_train.isnull().sum()

doi                   0
title                 0
abstract              0
authors               0
venue               403
year                  0
references            0
topics             1940
is_open_access        0
fields_of_study       0
citations             0
dtype: int64

In [16]:
mode_venue = df_train["venue"].mode().values[0]
mode_venue

'ACL'

In [17]:
mode_topics = df_train["topics"].mode().values[0]
mode_topics

'Neural machine translation'

In [18]:
df_train["venue"] = df_train["venue"].fillna(mode_venue)

In [19]:
df_train["topics"] = df_train["topics"].fillna(mode_topics)

In [20]:
df_train.isnull().sum()

doi                0
title              0
abstract           0
authors            0
venue              0
year               0
references         0
topics             0
is_open_access     0
fields_of_study    0
citations          0
dtype: int64

In [21]:
df_test["authors"] = [",".join(i) for i in df_test["authors"]]
df_test["topics"] = [",".join(i) for i in df_test["topics"]]
df_test["fields_of_study"] = [",".join(i) for i in df_test["fields_of_study"]]

In [22]:
df_test = df_test.replace(r'^\s*$', np.nan, regex=True)

In [23]:
df_test.isnull().sum()

doi                  0
title                0
abstract             0
authors              0
venue               50
year                 0
references           0
topics             209
is_open_access       0
fields_of_study      0
dtype: int64

In [24]:
mode_v = df_test["venue"].mode().values[0]
mode_v

'ACL'

In [25]:
mode_top = df_test["topics"].mode().values[0]
mode_top

'Named-entity recognition'

In [26]:
df_test["venue"] = df_test["venue"].fillna(mode_v)

In [27]:
df_test["topics"] = df_test["topics"].fillna(mode_top)

In [28]:
df_test.isnull().sum()

doi                0
title              0
abstract           0
authors            0
venue              0
year               0
references         0
topics             0
is_open_access     0
fields_of_study    0
dtype: int64

In [29]:
df_train.shape

(9381, 11)

In [30]:
df_test.shape

(968, 10)

In [31]:
df_train["year"] = df_train["year"].astype(int)

In [32]:
df_combined = df_train.append(df_test, ignore_index=True)

In [33]:
df_combined.head()

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",ACL,2015,39,Semantic role labeling,True,Computer Science,60.0
1,10.18653/v1/2020.eval4nlp-1.12,One of these words is not like the other: a re...,Word embeddings are an active topic in the NLP...,"Jesper Brink Andersen,Mikkel Bak Bertelsen,Mik...",EVAL4NLP,2020,44,Neural machine translation,True,Computer Science,1.0
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",INLG,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5.0
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",SemEval@ACL,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5.0
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",IWPT,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10.0


In [34]:
duplicates = df_combined.duplicated()
print(duplicates.sum())

1


In [35]:
df_combined.drop_duplicates(inplace=True)

In [36]:
d_check = df_combined.duplicated()
print(d_check.sum())

0


In [37]:
def clean_venue(text):
    n = re.sub(r'[0-9]', "", text)
    p = re.sub(r'[^\w\s]', "", n.lower().strip())
    return p

In [38]:
df_combined["venue"] = df_combined["venue"].apply(lambda x: clean_venue(x))

In [39]:
df_combined.head()

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",acl,2015,39,Semantic role labeling,True,Computer Science,60.0
1,10.18653/v1/2020.eval4nlp-1.12,One of these words is not like the other: a re...,Word embeddings are an active topic in the NLP...,"Jesper Brink Andersen,Mikkel Bak Bertelsen,Mik...",evalnlp,2020,44,Neural machine translation,True,Computer Science,1.0
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",inlg,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5.0
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5.0
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10.0


### One-Hot Encoding Categorical Variables
venue, year,is_open_access, fields_of_study

In [40]:
df_combined = pd.get_dummies(df_combined, columns=["venue", "year", "is_open_access", "fields_of_study"])

In [41]:
df_combined.shape

(10348, 368)

In [42]:
df_combined.head()

,doi,title,abstract,authors,references,topics,citations,venue_acl,venue_acl workshop on natural language processing in the biomedical domain,venue_aclijcnlp,...,"fields_of_study_Medicine,Psychology",fields_of_study_Philosophy,"fields_of_study_Philosophy,Computer Science",fields_of_study_Political Science,"fields_of_study_Political Science,Computer Science",fields_of_study_Psychology,"fields_of_study_Psychology,Computer Science","fields_of_study_Psychology,Medicine",fields_of_study_Sociology,"fields_of_study_Sociology,Computer Science"
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",39,Semantic role labeling,60.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.18653/v1/2020.eval4nlp-1.12,One of these words is not like the other: a re...,Word embeddings are an active topic in the NLP...,"Jesper Brink Andersen,Mikkel Bak Bertelsen,Mik...",44,Neural machine translation,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",30,"Natural language generation,Library (computing...",5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",11,"Parsing,Convolutional neural network,Text-base...",5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",26,"Textual entailment,Parsing,SemEval,Semantic We...",10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Cleaning Text Data
1.Lowercase,removing punctuations, numbers and special characters\
2.Removing stop words\
3.Tokenization\
4.Stemming or Lemmatization

In [43]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
Stopwords = set(stopwords.words('english'))

In [46]:
def clean_text(text):
    text_clean = "".join([word.lower() for word in text if word not in string.punctuation])
    text_rc = re.sub('[0-9]+', '', text_clean)
    tokens = re.split('\W+', text_rc)
    text = [word for word in tokens if word not in Stopwords]
    return text

In [47]:
df_combined["title"] = df_combined["title"].apply(lambda x: clean_text(x))

In [48]:
df_combined["abstract"] = df_combined["abstract"].apply(lambda x: clean_text(x))

In [49]:
authors = df_combined[["authors"]]
authors['authors_k'] = authors['authors'].apply(lambda x: " ".join(x.lower() for x in x.split()))
authors['authors_p'] = [x.replace("."," ").split(',') for x in authors['authors_k']]
df_combined["authors"] = authors['authors_p']

In [50]:
topics = df_combined[["topics"]]
topics['topics_k'] = topics['topics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
topics['topics_p'] = [x.replace(" ", ",").replace("(","").replace(")","").split(',') for x in topics['topics_k']]
df_combined["topics"] = topics["topics_p"]

In [51]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [52]:
from nltk.stem import PorterStemmer

In [53]:
ps = PorterStemmer()

In [54]:
df_combined["title"] = df_combined["title"].apply(lambda x:[ps.stem(i) for i in x])

In [55]:
df_combined["abstract"] = df_combined["abstract"].apply(lambda x:[ps.stem(i) for i in x])

In [56]:
df_combined["topics"] = df_combined["topics"].apply(lambda x:[ps.stem(i) for i in x])

In [57]:
df_combined["title"] = [" ".join(i) for i in df_combined["title"]]

In [58]:
df_combined["abstract"] = [" ".join(i) for i in df_combined["abstract"]]

In [59]:
df_combined["authors"] = [" ".join(i) for i in df_combined["authors"]]

In [60]:
df_combined["topics"] = [" ".join(i) for i in df_combined["topics"]]

In [61]:
df_combined.head()

,doi,title,abstract,authors,references,topics,citations,venue_acl,venue_acl workshop on natural language processing in the biomedical domain,venue_aclijcnlp,...,"fields_of_study_Medicine,Psychology",fields_of_study_Philosophy,"fields_of_study_Philosophy,Computer Science",fields_of_study_Political Science,"fields_of_study_Political Science,Computer Science",fields_of_study_Psychology,"fields_of_study_Psychology,Computer Science","fields_of_study_Psychology,Medicine",fields_of_study_Sociology,"fields_of_study_Sociology,Computer Science"
0,10.3115/v1/P15-1039,gener high qualiti proposit bank multilingu se...,semant role label srl crucial natur languag un...,a akbik laura chiticariu marina danilevsky yu...,39,semant role label,60.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.18653/v1/2020.eval4nlp-1.12,one word like reproduct outlier identif use no...,word embed activ topic nlp research commun sta...,jesper brink andersen mikkel bak bertelsen mik...,44,neural machin translat,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.18653/v1/W17-3516,codetext challeng text gener sourc librari,propos new share task tactic datatotext gener ...,kyle richardson sina zarrieß jonas kuhn,30,natur languag gener librari comput program lan...,5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.18653/v1/S17-2160,mean factori semev task produc amr neural sema...,evalu semant parser base characterbas sequence...,rik van noord johan bos,11,pars convolut neural network text-bas comput a...,5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10.18653/v1/W15-2205,semant pars textual entail,paper gaug util generalpurpos opendomain seman...,elisabeth lien milen kouylekov,26,textual entail pars semev semant web rule lang...,10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Split the data back into train and test

In [62]:
df_train = df_combined.loc[:9380,]

In [63]:
df_test = df_combined.loc[9381:,]

In [64]:
X = df_train.loc[:,df_train.columns != 'citations']

In [65]:
y = df_train["citations"].astype(int)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(7504, 367) (1876, 367) (7504,) (1876,)


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
tfidf = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000)

In [92]:
abstract_train = tfidf.fit_transform(X_train["abstract"])

In [93]:
df_abs = pd.DataFrame(abstract_train.todense(), columns=tfidf.get_feature_names())

In [95]:
abstract_val = tfidf.transform(X_val["abstract"])

In [96]:
df_val = pd.DataFrame(abstract_val.todense(), columns=tfidf.get_feature_names())

In [98]:
X_train.drop(['title', 'abstract', 'authors', 'topics'], axis=1, inplace=True)

In [111]:
X_val.drop(['title', 'abstract', 'authors', 'topics'], axis=1, inplace=True)

In [110]:
X_train_final = pd.concat([X_train.reset_index(), df_abs.reset_index()], axis=1)

In [113]:
X_val_final = pd.concat([X_val.reset_index(), df_val.reset_index()], axis=1)

In [115]:
X_train_final.shape

(7504, 1365)

In [116]:
X_val_final.shape

(1876, 1365)

In [123]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_final = scaler.fit_transform(X_train_final.iloc[:, 2:])
X_val_final = scaler.transform(X_val_final.iloc[:, 2:])

In [135]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100,random_state=0)

regressor.fit(X_train_final, y_train)

RandomForestRegressor(random_state=0)

In [136]:
pred = regressor.predict(X_val_final)

In [137]:
from sklearn.metrics import r2_score

r2_score(y_val,pred)

-0.9904344614933578

In [138]:
def score(Y_true, Y_pred):
    y_true = np.log1p(np.maximum(0, Y_true))
    y_pred = np.log1p(np.maximum(0, Y_pred))
    return 1 - np.mean((y_true-y_pred)**2) / np.mean((y_true-np.mean(y_true))**2)

def evaluate(gold_path, pred_path):
    gold = { x['doi']: x['citations'] for x in json.load(open(gold_path)) }
    pred = { x['doi']: x['citations'] for x in json.load(open(pred_path)) }
    y_true = np.array([ gold[key] for key in gold ])
    y_pred = np.array([ pred[key] for key in gold ])
    return score(y_true, y_pred)

In [139]:
score(y_val, pred)

-0.13340345588840674

In [133]:
from sklearn.metrics import mean_squared_error

In [140]:
mean_squared_error(y_val,pred)

16279.54804152452

In [141]:
from sklearn.linear_model import LinearRegression

In [142]:
lr = LinearRegression()

In [143]:
lr.fit(X_train_final,y_train)

LinearRegression()

In [144]:
y_pred = lr.predict(X_val_final)

In [146]:
r2_score(y_val,y_pred)

-6.600904852304667e+22

In [147]:
mean_squared_error(y_val,y_pred)

5.3988086389944545e+26